In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # showing and rendering figures
# from skimage.io import imread
import os
from glob import glob
import tensorflow as tf

IMG_SIZE = 512  # slightly smaller than vgg16 normally expects
batch_size = 4

config = tf.compat.v1.ConfigProto(gpu_options =
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

base_image_dir = os.path.join('D:\\Datasets\\aptos2019-blindness-detection')

retina_df = pd.read_csv(os.path.join(base_image_dir, 'train.csv'))
retina_df['path'] = retina_df['id_code'].map(lambda x: os.path.join(base_image_dir,'train_images',
                                                         '{}.png'.format(x)))
retina_df['exists'] = retina_df['path'].map(os.path.exists)
print(retina_df['exists'].sum(), 'images found of', retina_df.shape[0], 'total')

retina_df.dropna(inplace = True)
retina_df = retina_df[retina_df['exists']].drop_duplicates()
retina_df['level'] = retina_df['diagnosis'].map(lambda x: str(x))

# retina_df['diagnosis'].hist(figsize = (10, 5))
# plt.show()

########################### Split Data into Training and Validation ############################
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
raw_df = retina_df.groupby(['diagnosis']).apply(lambda x: x.sample(n = 2000, replace = True)
                                                      ).reset_index(drop = True)
# raw_df.drop_duplicates()
# a = raw_df.groupby('id_code').count()>1
# price = a[a['diagnosis'] == True].index
# repeat_df = raw_df[raw_df['id_code'].isin(price)]   #show duplicated elements

print('New Data Size:', raw_df.shape[0], 'Old Size:', retina_df.shape[0])

train_df, valid_df = train_test_split(raw_df,
                                   test_size = 0.2,
                                   random_state = 2018)
print('train', train_df.shape[0], 'validation', valid_df.shape[0])

raw_df['diagnosis'].hist(figsize = (10, 5))
plt.show()



#################################################################################################
from tensorflow.keras.applications.inception_v3 import preprocess_input
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20, horizontal_flip=True,
    vertical_flip=True, preprocessing_function=preprocess_input,
    data_format="channels_last")
train_generator = train_datagen.flow_from_dataframe(train_df, x_col='path', y_col='level',
                                                    target_size=(IMG_SIZE,IMG_SIZE),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    class_mode='categorical')

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=0, horizontal_flip=False,
    vertical_flip=False, preprocessing_function=preprocess_input,
    data_format="channels_last")
valid_generator = valid_datagen.flow_from_dataframe(valid_df, x_col='path', y_col='level',
                                                    target_size=(IMG_SIZE,IMG_SIZE),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')
print(train_generator)
t_x, t_y = next(train_generator)
input_shape=t_x.shape[1:]
num_classes=t_y.shape[1]
print(t_x.shape)
print(t_y.shape)

import ResidualAttentionNetwork as RAN
from tensorflow.keras import optimizers

epochs = 50
weight_path="{}_weights_best.hdf5".format('retina')
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1,
                             save_best_only=True, mode='auto', save_weights_only=True)

model = RAN.RANetwork((IMG_SIZE, IMG_SIZE, 3), 5, activation='softmax').build_model()
model.summary()
model.compile(optimizer=optimizers.Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=train_df.shape[0]//batch_size,
                              verbose=1,
                              validation_data=valid_generator,
                              validation_steps=valid_df.shape[0]//batch_size,
                              epochs=epochs)